In [89]:
from langchain_openai import AzureChatOpenAI
import os
import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers.string import StrOutputParser

In [ ]:
# Erstellung der LLM Instanz
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",  # or your deployment
    api_version="2025-01-01-preview",  # or your api version
    temperature=0.2,
    max_tokens=16000,
    timeout=None)

Meine Idee: 
- Die Idee ist es jedes einzelne Teilsystem der Systemarchitektur durchzugehen und Anforderunngen zu generieren
- Alles innerhalb einer Langchain Chain
- Die Input deine Templates ist das System der Ebene 1 und das System der Ebene 2 und er kreiiert die Anforderungen für das Ebene 3 System


In [20]:
df=pd.read_json("./input_data/full_architecture.json")

In [22]:
df.head()

,name,systems
0,Gesamtfahrzeug,"{'name': 'Fahrsystem', 'subsystems': [{'name':..."
1,Gesamtfahrzeug,"{'name': 'Energiesystem', 'subsystems': [{'nam..."
2,Gesamtfahrzeug,"{'name': 'Bodysystem', 'subsystems': [{'name':..."
3,Gesamtfahrzeug,"{'name': 'Infotainmentsystem', 'subsystems': [..."


In [50]:
df.iloc[0,1]['name'],df.iloc[0,1]['subsystems'][1]['name'],df.iloc[0,1]['subsystems'][1]['subsystems']

('Fahrsystem',
 'Drehmomentverteilungssystem',
 ['Torque-Vectoring-System',
  'Achsschlupfregelsystem',
  'Allradkoordinationssystem',
  'Drehmomentpriorisierungssystem',
  'Drehmomentdiagnosesystem'])

In [ ]:
df.iloc[0,1]['subsystems'][0]['subsystems']

Vorderachsantriebssystem
Hinterachsantriebssystem
Leistungsverteilungssystem
Antriebssteuerungssystem
Getriebemanagementsystem


# Langchain

In [95]:
kontext="""Initial Anforderungen BEV-Fahrzeug SUV, Gewicht 2,5 t Beschleunigung 0-100 in 3 sec, Batteriekapazität 110 KW, Ladegeschwindigkeit 400 KW/verbrauch 14 KW /100km, Produktion in USA Tuscaloosa, 300000 Einheiten pro Jahr
Bremsen von 100 auf 0 in 30 Metern, Infotainment mit Navi, 3 Telefone Parallel, Teilautonomes fahren Level3, Komfortfahrwerk mit Luftfederung und digitalen Einstellungen,
Alle anderen Anforderungen auf Basis EQE500.

Anforderungen Design: Mergen zwischen EQE 500 und BMW X5 sowie Rangrover Sport"""

In [94]:
req_generator = PromptTemplate(
    input_variables=["vehicle_context", "level_1", "level_2", "level_3"],
    template="""
Du bist ein erfahrener Komponentenarchitekt mit 15 Jahren Berufserfahrung im Premium-Automotive-Bereich (BEV, SUV, teilautonomes Fahren, Hochvoltarchitektur).

Deine Aufgabe: Formuliere **realistische funktionale und nicht-funktionale Anforderungen** an ein Fahrzeugsystem auf Ebene 3, unter Berücksichtigung der übergeordneten Fahrzeug- und Systemkontexte.

---

**Fahrzeugkontext**:
{vehicle_context}

**Systemstruktur**:
- Ebene 1: {level_1}
- Ebene 2: {level_2}
- Ebene 3 (Zielkomponente): {level_3}

---

**Deine Antwortstruktur (bitte exakt einhalten)**:

Funktionale Anforderungen:
- ...
- ...

Nicht-funktionale Anforderungen:
- ...
- ...

Ergänze ggf. Annahmen, falls für die Antwort relevant.
"""
)

In [123]:
chain= req_generator|llm|StrOutputParser()

In [125]:
chain.invoke({"vehicle_context":kontext,"level_1":"Fahrsystem","level_2":"Antriebssystem","level_3":"Bremssystem"})

'### Annahmen:\n1. Das Bremssystem ist Teil eines integrierten Fahrzeugsystems und muss mit anderen Subsystemen wie Antrieb, Fahrwerk und Fahrerassistenzsystemen (ADAS) interagieren.\n2. Das Fahrzeug ist für den Premium-Markt konzipiert, daher sind hohe Anforderungen an Komfort, Sicherheit und Performance zu berücksichtigen.\n3. Das Bremssystem muss sowohl für manuelles als auch für teilautonomes Fahren (Level 3) optimiert sein.\n4. Produktionsstandort USA (Tuscaloosa) erfordert Berücksichtigung von lokalen Vorschriften und Standards (z. B. FMVSS, SAE).\n\n---\n\n### Funktionale Anforderungen:\n1. **Bremsleistung**: Das Bremssystem muss in der Lage sein, das Fahrzeug aus einer Geschwindigkeit von 100 km/h auf 0 km/h innerhalb von maximal 30 Metern bei voller Beladung (2,5 t) zu stoppen.\n2. **Integration mit ADAS**: Das Bremssystem muss nahtlos mit dem teilautonomen Fahrmodus (Level 3) interagieren, um automatisierte Bremsmanöver bei Gefahrensituationen auszuführen.\n3. **Regenerative 

In [128]:
answer_txt=""

In [129]:
for index,row in df.iterrows():
    level_1=row["systems"]["name"]

    for x in range(len(row["systems"]['subsystems'])):
        level_2=row["systems"]['subsystems'][x]['name']
        for level_3 in row["systems"]['subsystems'][x]['subsystems']:
            answer_txt+=(f"Anforderungen an {level_3}\n")
            llm_output=chain.invoke({"vehicle_context":kontext,"level_1":level_1,"level_2":level_2,"level_3":level_3})
            answer_txt+=llm_output
            answer_txt+="\n"
            
            


In [131]:
with open("./output_data/anforderungen.txt", "a") as f:
  f.write(answer_txt)



In [130]:
answer_txt

'Anforderungen an Vorderachsantriebssystem\n### Annahmen:\n1. Das Vorderachsantriebssystem ist Teil des elektrischen Antriebsstrangs und umfasst Elektromotor, Leistungselektronik, Kühlung und mechanische Komponenten wie Achsanbindung und Differential.\n2. Das System muss sowohl die Anforderungen an Performance (Beschleunigung, Effizienz) als auch an Komfort und Sicherheit erfüllen.\n3. Die Produktion erfolgt in Tuscaloosa, USA, mit einer Jahresstückzahl von 300.000 Einheiten, was Skalierbarkeit und Fertigungsoptimierung erfordert.\n4. Das Fahrzeug ist ein BEV-SUV mit Premium-Anspruch, daher sind hohe Qualitätsstandards und Langlebigkeit essenziell.\n\n---\n\n### Funktionale Anforderungen:\n1. **Leistungsbereitstellung**: Das Vorderachsantriebssystem muss eine maximale Leistung von mindestens 200 kW liefern, um die Gesamtbeschleunigung von 0-100 km/h in 3 Sekunden zu unterstützen.\n2. **Effizienz**: Das System muss einen Wirkungsgrad von mindestens 95 % bei typischen Fahrbedingungen gew